Code extracts text from pdf files in directory in recursion
and writes txt file in the same location

In [1]:
# install
# pip install pdfminer.six
# pip install msgpack

In [2]:
import os
from pathlib import Path

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO

In [3]:
def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()
    page_number = 0
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
        page_number += 1
    fp.close()
    device.close()
    str = retstr.getvalue()
    retstr.close()
    return str, page_number

In [4]:
def write_text_to_file(path_in_str, s):
    basename = os.path.splitext(path_in_str)[0]
    file_to_write = os.path.join(basename + ".frompdf.txt")
    f = open(file_to_write, 'wb')
    f.write(s.encode('utf-8'))
    f.close()

In [5]:
def process_all_files(directory_in_str):
    pathlist = Path(directory_in_str).glob('**/*.pdf')
    file_count = 0
    for path in pathlist:
        # because path is object not string
        path_in_str = str(path)
        print("Procesing file %s" % path_in_str)
        
        s, page_count = convert_pdf_to_txt(path_in_str)
        if page_count > 0:
            write_text_to_file(path_in_str, s)
            print("Done file %s. Pages: %s" % (path_in_str, page_count))
        else:
            print("-----> Check file %s" % path_in_str)

In [6]:
directory_in_str = r"d:\dir_with_files"
process_all_files(directory_in_str)

Procesing file d:\dir_with_files\Elasticsearch Indexing.pdf
Done file d:\dir_with_files\Elasticsearch Indexing.pdf. Pages: 176
Procesing file d:\dir_with_files\another_dir\Elasticsearch Blueprints.pdf
Done file d:\dir_with_files\another_dir\Elasticsearch Blueprints.pdf. Pages: 192
